In [1]:
import pandas as pd
import tensorflow as tf
import keras as ks
import numpy as np
from sklearn.metrics import recall_score, precision_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import datetime
import time
from math import floor
import models.scripts.logistic_regression_functions as lrf
import models.scripts.helper_functions as hf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, Flatten, Activation, MaxPooling2D
import matplotlib.pylab as plt
import seaborn as sns

C:\Users\mccomackjp\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Load the data
df = pd.read_csv('../../data/cleaned/site1_vineyard.csv')
df.dtypes

FileNotFoundError: File b'../../data/cleaned/site1_vineyard.csv' does not exist

In [ ]:
# Create a single datetime column from the strings provided in our csv's
# (We're already doing this bit for our models)
from datetime import datetime

timestamp = df['Date (mm.dd.yyyy)'] + ' '+ df['Time 24hr']
timestamp = pd.to_datetime(timestamp)
df['timestamp'] = timestamp

# I'll drop the other columns for simple demo purposes
df = df[['timestamp', 'pH', 'Temp C']]
df['temp'] = [x for x in range(0, df.shape[0])]

df.dtypes

In [ ]:
# Now we set our datetime to be the index of the df for awesome indexing options
# Note, the column is removed, so it might be good to keep a copy by another
# name for feature engineering.
df['datetime'] = df['timestamp']
df = df.set_index('timestamp')
df.dtypes


In [ ]:
# Index into a range of dates
df['2017-05-05 00:15:00':'2017-05-05 00:45:00']

In [ ]:
# Index with an offset
start = df['datetime'][0]
delta = pd.Timedelta(1, unit='h')
print("start:", start)
print(delta)
offset = pd.Timedelta(1, 'm')
# grab a slice over 1 hour
df[start:start + delta]

In [ ]:
df[start+delta+offset: start+delta+delta]

In [ ]:
df[start:start + delta].describe()

In [ ]:
df[start:start + delta].quantile(.5, numeric_only=False)['datetime']

In [ ]:
test = df[start:start + delta]['temp']
print("0%", test.quantile(0.0))
print("25%", test.quantile(0.25))
print("50%", test.quantile(0.5))
print("75%", test.quantile(0.75))
print("100%", test.quantile(1))
df[start:start + delta].describe()

In [ ]:
temp = df[start:start + delta]
a = temp[0:0]
a = a.append(temp.quantile(.5, numeric_only=False))

In [ ]:
dfs = [df[start:start + delta], df[start+delta+offset: start+delta+delta]]


In [ ]:
# extracted = dfs[0][0:0] # start with a blank data frame with the same setup
extracted = pd.DataFrame()
for d in dfs:
    extracted = extracted.append(d.quantile(.5, numeric_only=False))
extracted['datetimeIndex'] = extracted['datetime']
extracted = extracted.set_index('datetimeIndex')

In [ ]:
extracted

In [ ]:
pd.date_range(start='2018-01-01', periods=10, freq='d')

In [ ]:
df['datetime'][len(df['datetime'])-1]

In [ ]:
start = df['datetime'][0]
df[start:start + pd.Timedelta(1, 'm')]

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
print(type(df.quantile(.5)))
df['datetime'].quantile(.5)

In [ ]:
def segment(df, time_col, x_win, y_win, shift):
    segments = []
    targets = []
    start = df[time_col][0]
    end = df[time_col][len(df[time_col])-1]
    offset = pd.Timedelta(1, unit='s') # to remove overlap of x and y
    while start + x_win + y_win <= end:
        segments.append(df[start:start + x_win])
        targets.append(df[start  + x_win + offset : start + x_win + y_win])
        start += shift
    return segments, targets
        

In [ ]:
s, t = segment(df, 
               'datetime', 
               pd.Timedelta(2, unit='d'), pd.Timedelta(1, unit='d'),
        pd.Timedelta(6, unit='h'))

In [ ]:
sum(range(0,10))/10

In [ ]:
[i for i in range(0, 10)]

## Segment categories tests

In [ ]:
s, t = 